In [183]:
import gspread
import pandas as pd
import requests
import geopy.distance
import numpy as np
from datetime import timedelta
import time
import json
from itertools import chain

In [184]:
gc = gspread.oauth()

In [185]:
sh = gc.open_by_key('1bKaLOU0vsS114xZVPG8bRNQT2lWiPsqj7HJ7N53z4FM')

In [186]:
ws = sh.worksheet("Página1")

In [187]:
df = pd.DataFrame(ws.get_all_records())

In [188]:
df['Latitude'] = df['Latitude'].astype("string")
df['Longitude'] = df['Longitude'].astype("string")

In [189]:
df['Latitude'] = df['Latitude'].replace("@", ".", regex=True)
df['Longitude'] = df['Longitude'].replace("@", ".", regex=True)

In [190]:
exluirLinhas = df[df['Longitude']=='NULL'].index
df=df.drop(exluirLinhas)
exluirLinhas = df[df['Latitude']=='NULL'].index
df=df.drop(exluirLinhas)

In [191]:
df.dtypes

DataHoraPassagem         object
PlacaVeiculo             object
VelocidadeMedida          int64
VelocidadeConsiderada     int64
Latitude                 string
Longitude                string
NumeroSerie              object
Logradouro               object
Sentido                  object
Bairro                   object
Nome                     object
TipoInfracao             object
PLACA SILVER             object
dtype: object

In [192]:
df.head()

,DataHoraPassagem,PlacaVeiculo,VelocidadeMedida,VelocidadeConsiderada,Latitude,Longitude,NumeroSerie,Logradouro,Sentido,Bairro,Nome,TipoInfracao,PLACA SILVER
0,2022-03-10 00:00:35.000,OMK6G21,58,51,-19.9272265,-43.9334856,BRI1726,AV. AFONSO PENA,FAIXA 1 - ESQUERDA,Funcionários,CONJUGADO,Velocidade,AHN2851
1,2022-03-10 00:00:35.000,OMK6G21,58,51,-19.9272265,-43.9334856,BRI1726,AV. AFONSO PENA,FAIXA 1 - ESQUERDA,Funcionários,CONJUGADO,CirculacaoRestrita,AKP5579
2,2022-03-10 00:00:35.000,OMK6G21,58,51,-19.9272265,-43.9334856,BRI1726,AV. AFONSO PENA,FAIXA 1 - ESQUERDA,Funcionários,CONJUGADO,ParadaFaixa,AMO8396
3,2022-03-10 00:00:35.000,OMK6G21,58,51,-19.9272265,-43.9334856,BRI1726,AV. AFONSO PENA,FAIXA 1 - ESQUERDA,Funcionários,CONJUGADO,FaixaExclusiva,AOD8686
4,2022-03-10 00:00:35.000,OMK6G21,58,51,-19.9272265,-43.9334856,BRI1726,AV. AFONSO PENA,FAIXA 1 - ESQUERDA,Funcionários,CONJUGADO,ConversaoProibida,AQD8686


In [193]:
df_directions = df[['NumeroSerie','Latitude','Longitude']].drop_duplicates()

In [194]:
listagem_base = df_directions.values.tolist()
listagem_base

[['BRI1726', '-19.9272265', '-43.9334856'],
 ['BRI1286', '-19.9177640', '-43.9347130'],
 ['BRI1536', '-19.9041940', '-43.9323060'],
 ['BRI1140', '-19.8763330', '-43.9283570'],
 ['BRI1292', '-19.9177340', '-43.9346980'],
 ['BRI1263', '-19.8760650', '-43.9287370'],
 ['BRI1543', '-19.9034440', '-43.9326940'],
 ['BRI1265', '-19.8226740', '-43.9448640'],
 ['BRI1522', '-19.8276110', '-43.9600830'],
 ['BRI1582', '-19.8862220', '-43.9289170'],
 ['BRI1700', '-19.9144500', '-43.9362920'],
 ['BRI1356', '-19.9175030', '-43.9506160'],
 ['BRI1544', '-19.9034440', '-43.9326940'],
 ['BRI1376', '-19.9542140', '-43.9856280'],
 ['BRI1118', '-19.8571311', '-43.9600656'],
 ['BRI1683', '-19.9147060', '-43.9360470'],
 ['BRI1039', '-19.8773720', '-43.9291190'],
 ['BRI1313', '-19.9084640', '-43.9626530'],
 ['BRI1539', '-19.9034440', '-43.9326940'],
 ['BRI1535', '-19.8858330', '-43.9295000'],
 ['BRI1030', '-19.8885274', '-43.9294711'],
 ['BRI1506', '-19.8276390', '-43.9598610'],
 ['BRI1369', '-19.9469889', '-43

In [265]:
df_gold=[]
list_infos=[]

#API GOOGLE
#Como pegar dados do google api 
url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=45.7597%2C4.8422&destinations=48.8567%2C2.3508&key="

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

res_json = (response.text)
res_final = json.loads(res_json)

EnderecoDestino = res_final['destination_addresses'][0]
EnderecoOrigem = res_final['origin_addresses'][0]
Distance_text = res_final["rows"][0]['elements'][0]['distance']['text']
Distance_value = res_final["rows"][0]['elements'][0]['distance']['value']
durantion_text = res_final["rows"][0]['elements'][0]['duration']['text']
durantion_value = res_final["rows"][0]['elements'][0]['duration']['value']

print(EnderecoDestino)
print(EnderecoOrigem)
print(Distance_text)
print(Distance_value)
print(durantion_text)
print(durantion_value)

list_infos = [EnderecoDestino,EnderecoOrigem,Distance_text,Distance_value,durantion_text,durantion_value]

print(list_infos)

In [264]:
#API GOOGLE
def buscaGoogle(geoOrigem,geoDestino):
    
    print("Sera tratado origem: "+str(geoOrigem))
    print("Sera tratado destino: "+str(geoDestino))
    
    list_infos = []
    
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins="+geoOrigem[0]+"%2C"+geoOrigem[1]+"&destinations="+geoDestino[0]+"%2C"+geoDestino[1]+"&key="
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    res_json = (response.text)
    res_final = json.loads(res_json)
    
    verifica_status = res_final["rows"][0]['elements'][0]['status']
    
    print(verifica_status)
    
    print(res_final)
    print("\n-------------------")

    EnderecoDestino = res_final['destination_addresses'][0]
    EnderecoOrigem = res_final['origin_addresses'][0]
    Distance_text = res_final["rows"][0]['elements'][0]['distance']['text']
    Distance_value = res_final["rows"][0]['elements'][0]['distance']['value']
    durantion_text = res_final["rows"][0]['elements'][0]['duration']['text']
    durantion_value = res_final["rows"][0]['elements'][0]['duration']['value']
    
    list_infos = [EnderecoDestino,EnderecoOrigem,Distance_text,Distance_value,durantion_text,durantion_value]
    
    #time.sleep(1)
    
    return list_infos

In [266]:
def CalcListagem(radar, lat,long):
    for i in listagem_base:
        if i[0] != radar:
            geoOrigem=[lat,long]
            geoDestino=[i[1],i[2]]
            dis = buscaGoogle(geoOrigem,geoDestino)
            if dis != 0 and dis is not None:
                df_gold.append(
                    {
                        'RadarBase': radar,
                        'LatitudeBase': lat,
                        'LongitudeBase':  long,
                        'RadarComparacao':i[0],
                        'LatitudeComparacao':i[1],
                        'LongitudeComparacao':i[2],
                        'EnderecoDestino':dis[0],
                        'EnderecoOrigem':dis[1],
                        'DistanciaTexto':dis[2],
                        'Distanciavalor':dis[3],
                        'DuracaoTexto':dis[4],
                        'DuracaovAlor':dis[5]
                    }
                            )
                #print("RadarBase: "+str(df_gold))
                #print("\n")
    return True

In [224]:
df_gold

[{'RadarBase': 'BRI1726',
  'LatitudeBase': '-19.9272265',
  'LongitudeBase': '-43.9334856',
  'RadarComparacao': 'BRI1286',
  'LatitudeComparacao': '-19.9177640',
  'LongitudeComparacao': '-43.9347130',
  'EnderecoDestino': 'Av. dos Andradas, 339 - Centro, Belo Horizonte - MG, 30120-010, Brazil',
  'EnderecoOrigem': 'Av. Afonso Pena, 1707 - Centro, Belo Horizonte - MG, 30130-006, Brazil',
  'DistanciaTexto': '1.4 km',
  'Distanciavalor': 1438,
  'DuracaoTexto': '5 mins',
  'DuracaovAlor': 274},
 {'RadarBase': 'BRI1726',
  'LatitudeBase': '-19.9272265',
  'LongitudeBase': '-43.9334856',
  'RadarComparacao': 'BRI1536',
  'LatitudeComparacao': '-19.9041940',
  'LongitudeComparacao': '-43.9323060',
  'EnderecoDestino': 'R. Jacuí, 1036 A - Floresta, Belo Horizonte - MG, 31110-050, Brazil',
  'EnderecoOrigem': 'Av. Afonso Pena, 1707 - Centro, Belo Horizonte - MG, 30130-006, Brazil',
  'DistanciaTexto': '3.7 km',
  'Distanciavalor': 3747,
  'DuracaoTexto': '8 mins',
  'DuracaovAlor': 509},
 

In [267]:
for x in listagem_base:
    print(x)
    
    CalcListagem(x[0],x[1],x[2])
        
df_directions_Gold = pd.DataFrame(df_gold)

['BRI1726', '-19.9272265', '-43.9334856']
Sera tratado origem: ['-19.9272265', '-43.9334856']
Sera tratado destino: ['-19.9177640', '-43.9347130']
OK
{'destination_addresses': ['Av. dos Andradas, 339 - Centro, Belo Horizonte - MG, 30120-010, Brazil'], 'origin_addresses': ['Av. Afonso Pena, 1707 - Centro, Belo Horizonte - MG, 30130-006, Brazil'], 'rows': [{'elements': [{'distance': {'text': '1.4 km', 'value': 1438}, 'duration': {'text': '5 mins', 'value': 274}, 'status': 'OK'}]}], 'status': 'OK'}

-------------------
Sera tratado origem: ['-19.9272265', '-43.9334856']
Sera tratado destino: ['-19.9041940', '-43.9323060']
OK
{'destination_addresses': ['R. Jacuí, 1036 A - Floresta, Belo Horizonte - MG, 31110-050, Brazil'], 'origin_addresses': ['Av. Afonso Pena, 1707 - Centro, Belo Horizonte - MG, 30130-006, Brazil'], 'rows': [{'elements': [{'distance': {'text': '3.7 km', 'value': 3747}, 'duration': {'text': '8 mins', 'value': 506}, 'status': 'OK'}]}], 'status': 'OK'}

-------------------
S

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [268]:
df_directions_Gold = pd.DataFrame(df_gold)
df_directions_Gold

,RadarBase,LatitudeBase,LongitudeBase,RadarComparacao,LatitudeComparacao,LongitudeComparacao,EnderecoDestino,EnderecoOrigem,DistanciaTexto,Distanciavalor,DuracaoTexto,DuracaovAlor
0,BRI1726,-19.9272265,-43.9334856,BRI1286,-19.9177640,-43.9347130,"Av. dos Andradas, 339 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1438,5 mins,274
1,BRI1726,-19.9272265,-43.9334856,BRI1536,-19.9041940,-43.9323060,"R. Jacuí, 1036 A - Floresta, Belo Horizonte - ...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",3.7 km,3747,8 mins,506
2,BRI1726,-19.9272265,-43.9334856,BRI1140,-19.8763330,-43.9283570,"Av. Cristiano Machado, 5441 - União, Belo Hori...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",7.1 km,7130,13 mins,758
3,BRI1726,-19.9272265,-43.9334856,BRI1292,-19.9177340,-43.9346980,"Av. dos Andradas, 337 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1442,5 mins,271
4,BRI1726,-19.9272265,-43.9334856,BRI1263,-19.8760650,-43.9287370,Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,"Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",8.7 km,8665,16 mins,934
...,...,...,...,...,...,...,...,...,...,...,...,...
505,BRI1263,-19.8760650,-43.9287370,2200002,-19.8583251,-43.9596204,"Av. Pres. Antônio Carlos, 7362 - Lagoinha, Bel...",Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,6.3 km,6309,13 mins,772
506,BRI1263,-19.8760650,-43.9287370,2200003,-19.8619870,-43.9585989,Brt Move Campos Ufmg Belo Horizonte - Minas Ge...,Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,6.6 km,6557,14 mins,827
507,BRI1263,-19.8760650,-43.9287370,2200018,-19.9213220,-43.9243330,"R. Piauí, 8 - Santa Efigênia, Belo Horizonte -...",Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,7.7 km,7671,13 mins,774
508,BRI1263,-19.8760650,-43.9287370,2200019,-19.9199000,-43.9205500,"Av. dos Andradas. 3.000, loja 2077/2078 - Sant...",Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,8.5 km,8459,16 mins,953


In [304]:
df_realizado = df_directions_Gold[['RadarBase','LatitudeBase','LongitudeBase','RadarComparacao','LatitudeComparacao','LongitudeComparacao']]

listagem_realizado = df_realizado.values.tolist()

In [314]:
for base in listagem_realizado:
    if base[3]== 'BRI1286':
        
        print(base)
        print(base[0])
        print(base[3])

['BRI1726', '-19.9272265', '-43.9334856', 'BRI1286', '-19.9177640', '-43.9347130']
BRI1726
BRI1286
['BRI1536', '-19.9041940', '-43.9323060', 'BRI1286', '-19.9177640', '-43.9347130']
BRI1536
BRI1286
['BRI1140', '-19.8763330', '-43.9283570', 'BRI1286', '-19.9177640', '-43.9347130']
BRI1140
BRI1286
['BRI1292', '-19.9177340', '-43.9346980', 'BRI1286', '-19.9177640', '-43.9347130']
BRI1292
BRI1286
['BRI1263', '-19.8760650', '-43.9287370', 'BRI1286', '-19.9177640', '-43.9347130']
BRI1263
BRI1286


In [332]:
df_gold2=[]

In [336]:
def CalcListagem_fase2(radar, lat,long):
    for i in listagem_base:
        contador=0
        if i[0] != radar:
            for real in listagem_realizado:           
                if radar == real[0] and i[0] ==real[3]:
                    print(str(radar)+" e "+str(i[0]))
                    #print("Já foi inserido na lista")
                    contador=1
            if contador == 0:
                print("Não está na listagem, puxando do google")
                geoOrigem=[lat,long]
                geoDestino=[i[1],i[2]]

                dis = buscaGoogle(geoOrigem,geoDestino)
                if dis != 0 and dis is not None:
                    df_gold2.append(
                        {
                            'RadarBase': radar,
                            'LatitudeBase': lat,
                            'LongitudeBase':  long,
                            'RadarComparacao':i[0],
                            'LatitudeComparacao':i[1],
                            'LongitudeComparacao':i[2],
                            'EnderecoDestino':dis[0],
                            'EnderecoOrigem':dis[1],
                            'DistanciaTexto':dis[2],
                            'Distanciavalor':dis[3],
                            'DuracaoTexto':dis[4],
                            'DuracaovAlor':dis[5]
                        }
                                )
    return True

In [337]:
for x in listagem_base:
    CalcListagem_fase2(x[0],x[1],x[2])
        
df_directions_Gold2 = pd.DataFrame(df_gold2)

BRI1726 e BRI1286
BRI1726 e BRI1536
BRI1726 e BRI1140
BRI1726 e BRI1292
BRI1726 e BRI1263
BRI1726 e BRI1543
BRI1726 e BRI1265
BRI1726 e BRI1522
BRI1726 e BRI1582
BRI1726 e BRI1700
BRI1726 e BRI1356
BRI1726 e BRI1544
BRI1726 e BRI1376
BRI1726 e BRI1118
BRI1726 e BRI1683
BRI1726 e BRI1039
BRI1726 e BRI1313
BRI1726 e BRI1539
BRI1726 e BRI1535
BRI1726 e BRI1030
BRI1726 e BRI1506
BRI1726 e BRI1369
BRI1726 e BRI1028
BRI1726 e BRI1387
BRI1726 e BRI1353
BRI1726 e BRI1320
BRI1726 e BRI1128
BRI1726 e BRI1121
BRI1726 e BRI1119
BRI1726 e BRI1524
BRI1726 e 2200002
BRI1726 e 2200003
BRI1726 e 2200018
BRI1726 e 2200019
BRI1726 e 2200001
BRI1726 e 2200028
BRI1726 e 2200013
BRI1726 e 2200041
BRI1726 e 2200005
BRI1726 e 2200047
BRI1726 e 2200020
BRI1726 e 2200008
BRI1726 e 2200011
BRI1726 e BRI1402
BRI1726 e BRI1065
BRI1726 e BRI1201
BRI1726 e 2200006
BRI1726 e BRI1029
BRI1726 e BRI1362
BRI1726 e BRI1334
BRI1726 e BRI1673
BRI1726 e 2200017
BRI1726 e 2200010
BRI1726 e 2200004
BRI1726 e BRI1407
BRI1726 e 

In [339]:
df_directions_Gold2 = pd.DataFrame(df_gold2)

In [340]:
df_directions_Gold2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   RadarBase            55 non-null     int64 
 1   LatitudeBase         55 non-null     object
 2   LongitudeBase        55 non-null     object
 3   RadarComparacao      55 non-null     object
 4   LatitudeComparacao   55 non-null     object
 5   LongitudeComparacao  55 non-null     object
 6   EnderecoDestino      55 non-null     object
 7   EnderecoOrigem       55 non-null     object
 8   DistanciaTexto       55 non-null     object
 9   Distanciavalor       55 non-null     int64 
 10  DuracaoTexto         55 non-null     object
 11  DuracaovAlor         55 non-null     int64 
dtypes: int64(3), object(9)
memory usage: 5.3+ KB


In [341]:
df_directions_Gold2

,RadarBase,LatitudeBase,LongitudeBase,RadarComparacao,LatitudeComparacao,LongitudeComparacao,EnderecoDestino,EnderecoOrigem,DistanciaTexto,Distanciavalor,DuracaoTexto,DuracaovAlor
0,2200030,-19.8572390,-43.9601420,2200047,-19.8502170,-43.9645220,"Corredor MOVE Antônio Carlos, 10221 - Lagoinha...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",0.9 km,932,2 mins,118
1,2200030,-19.8572390,-43.9601420,2200020,-19.9538970,-43.9857940,"Av. Teresa Cristina, 7400 - Nova Cintra, Belo ...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",16.4 km,16442,26 mins,1537
2,2200030,-19.8572390,-43.9601420,2200008,-19.9579005,-43.9936091,"Av. Teresa Cristina, 9793 - Betânia, Belo Hori...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",16.4 km,16428,19 mins,1168
3,2200030,-19.8572390,-43.9601420,2200011,-19.8845908,-43.9726103,"Av. Presidente Carlos Luz 3000 - Pampulha, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",5.3 km,5258,8 mins,491
4,2200030,-19.8572390,-43.9601420,BRI1402,-19.9071780,-43.9435960,"Av. Pres. Antônio Carlos, 521 - Lagoinha, Belo...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",6.3 km,6339,10 mins,624
5,2200030,-19.8572390,-43.9601420,BRI1065,-19.8781300,-43.9289270,"Av. Cristiano Machado, 3060 - Vila Suzana, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",6.7 km,6694,16 mins,988
6,2200030,-19.8572390,-43.9601420,BRI1201,-19.8763150,-43.9283170,"Av. Cristiano Machado, 5441 - União, Belo Hori...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",6.6 km,6588,15 mins,902
7,2200030,-19.8572390,-43.9601420,2200006,-19.9539890,-43.9860330,"R. Benjamin Flôres, 252 - Nova Cintra, Belo Ho...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",17.5 km,17492,22 mins,1316
8,2200030,-19.8572390,-43.9601420,BRI1029,-19.8887256,-43.9291452,"Av. Cristiano Machado, 2116 - Cidade Nova, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",10.4 km,10445,18 mins,1064
9,2200030,-19.8572390,-43.9601420,BRI1362,-19.9469889,-43.9351541,"Av. Nossa Sra. do Carmo, 8 - São Pedro, Belo H...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",11.6 km,11598,23 mins,1403


In [347]:
frames = [result, df_directions_Gold2]

df_final = pd.concat(frames)

In [350]:
df_final.drop_duplicates()

,RadarBase,LatitudeBase,LongitudeBase,RadarComparacao,LatitudeComparacao,LongitudeComparacao,EnderecoDestino,EnderecoOrigem,DistanciaTexto,Distanciavalor,DuracaoTexto,DuracaovAlor
0,BRI1726,-19.9272265,-43.9334856,BRI1286,-19.9177640,-43.9347130,"Av. dos Andradas, 339 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1438,5 mins,274
1,BRI1726,-19.9272265,-43.9334856,BRI1536,-19.9041940,-43.9323060,"R. Jacuí, 1036 A - Floresta, Belo Horizonte - ...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",3.7 km,3747,8 mins,506
2,BRI1726,-19.9272265,-43.9334856,BRI1140,-19.8763330,-43.9283570,"Av. Cristiano Machado, 5441 - União, Belo Hori...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",7.1 km,7130,13 mins,758
3,BRI1726,-19.9272265,-43.9334856,BRI1292,-19.9177340,-43.9346980,"Av. dos Andradas, 337 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1442,5 mins,271
4,BRI1726,-19.9272265,-43.9334856,BRI1263,-19.8760650,-43.9287370,Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,"Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",8.7 km,8665,16 mins,934
...,...,...,...,...,...,...,...,...,...,...,...,...
50,2200030,-19.8572390,-43.9601420,2200029,-19.8570470,-43.9603330,"Av. Pres. Antônio Carlos - São Luiz, 7525 - La...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",2.2 km,2227,5 mins,303
51,2200030,-19.8572390,-43.9601420,BRI1408,-19.8379110,-43.9386690,"R. Francisco Bittencourt, 86 - Heliópolis, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",5.4 km,5381,11 mins,649
52,2200030,-19.8572390,-43.9601420,2200045,-19.8442170,-43.9662830,"Av. Dom Pedro I, 5550 - Pampulha, Belo Horizon...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",1.7 km,1678,3 mins,176
53,2200030,-19.8572390,-43.9601420,2200046,-19.8412470,-43.9678420,"Corredor MOVE Pedro I, 519 - Santa Branca, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",3.9 km,3888,8 mins,450


In [329]:
result.drop_duplicates()

,RadarBase,LatitudeBase,LongitudeBase,RadarComparacao,LatitudeComparacao,LongitudeComparacao,EnderecoDestino,EnderecoOrigem,DistanciaTexto,Distanciavalor,DuracaoTexto,DuracaovAlor
0,BRI1726,-19.9272265,-43.9334856,BRI1286,-19.9177640,-43.9347130,"Av. dos Andradas, 339 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1438,5 mins,274
1,BRI1726,-19.9272265,-43.9334856,BRI1536,-19.9041940,-43.9323060,"R. Jacuí, 1036 A - Floresta, Belo Horizonte - ...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",3.7 km,3747,8 mins,506
2,BRI1726,-19.9272265,-43.9334856,BRI1140,-19.8763330,-43.9283570,"Av. Cristiano Machado, 5441 - União, Belo Hori...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",7.1 km,7130,13 mins,758
3,BRI1726,-19.9272265,-43.9334856,BRI1292,-19.9177340,-43.9346980,"Av. dos Andradas, 337 - Centro, Belo Horizonte...","Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",1.4 km,1442,5 mins,271
4,BRI1726,-19.9272265,-43.9334856,BRI1263,-19.8760650,-43.9287370,Avenida Bernardo Vasconcelos 2700 Ipiranga Bel...,"Av. Afonso Pena, 1707 - Centro, Belo Horizonte...",8.7 km,8665,16 mins,934
...,...,...,...,...,...,...,...,...,...,...,...,...
8550,2200030,-19.8572390,-43.9601420,2200001,-19.8553235,-43.9611349,"Av. Pres. Antônio Carlos, 7719 - Lagoinha, Bel...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",2.0 km,2018,5 mins,282
8551,2200030,-19.8572390,-43.9601420,2200028,-19.8630310,-43.9579610,"Corredor MOVE Antônio Carlos, 2203 - Lagoinha,...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",2.3 km,2272,7 mins,404
8552,2200030,-19.8572390,-43.9601420,2200013,-19.8807138,-43.9758079,Av. Presidente Carlos Luz 3705 - Engenho Nogue...,"Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",4.3 km,4348,8 mins,491
8553,2200030,-19.8572390,-43.9601420,2200041,-19.9329560,-44.0084560,"R. Sen. Milton Campos, 202 - Tupi, Belo Horizo...","Av. Pres. Antônio Carlos - São Luiz, 7542 - La...",14.4 km,14370,18 mins,1083


In [330]:
df_realizado = result[['RadarBase','LatitudeBase','LongitudeBase','RadarComparacao','LatitudeComparacao','LongitudeComparacao']]

listagem_realizado = df_realizado.values.tolist()

In [331]:
df_realizado

,RadarBase,LatitudeBase,LongitudeBase,RadarComparacao,LatitudeComparacao,LongitudeComparacao
0,BRI1726,-19.9272265,-43.9334856,BRI1286,-19.9177640,-43.9347130
1,BRI1726,-19.9272265,-43.9334856,BRI1536,-19.9041940,-43.9323060
2,BRI1726,-19.9272265,-43.9334856,BRI1140,-19.8763330,-43.9283570
3,BRI1726,-19.9272265,-43.9334856,BRI1292,-19.9177340,-43.9346980
4,BRI1726,-19.9272265,-43.9334856,BRI1263,-19.8760650,-43.9287370
...,...,...,...,...,...,...
8550,2200030,-19.8572390,-43.9601420,2200001,-19.8553235,-43.9611349
8551,2200030,-19.8572390,-43.9601420,2200028,-19.8630310,-43.9579610
8552,2200030,-19.8572390,-43.9601420,2200013,-19.8807138,-43.9758079
8553,2200030,-19.8572390,-43.9601420,2200041,-19.9329560,-44.0084560


In [351]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9120 entries, 0 to 54
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   RadarBase            9120 non-null   object
 1   LatitudeBase         9120 non-null   object
 2   LongitudeBase        9120 non-null   object
 3   RadarComparacao      9120 non-null   object
 4   LatitudeComparacao   9120 non-null   object
 5   LongitudeComparacao  9120 non-null   object
 6   EnderecoDestino      9120 non-null   object
 7   EnderecoOrigem       9120 non-null   object
 8   DistanciaTexto       9120 non-null   object
 9   Distanciavalor       9120 non-null   int64 
 10  DuracaoTexto         9120 non-null   object
 11  DuracaovAlor         9120 non-null   int64 
dtypes: int64(2), object(10)
memory usage: 926.2+ KB


In [352]:
df_final.to_csv('bh_trans_distancias_google.csv', sep=';',encoding='utf-8')